In [1]:
from sklearn.base import BaseEstimator
import numpy as np

class MyDummyClassifier(BaseEstimator):
    def fit(self, X,y=None):
        pass

    def predict(self, X):
        pred = np.zeros((X.shape[0],1))
        for i in range(X.shape[0]): # sex ==1, survived=0으로 예측, 아니면 1로 예측측
            if X['Sex'].iloc[i] == 1:
                pred[i]=0
            else:
                pred[i]=1
        return pred


In [7]:
from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 피처 제거
def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return df

# 레이블 인코딩 수행.
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 데이터 전처리 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

titanic_df = pd.read_csv('./titanic_train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)

# 전처리리
X_titanic_df = transform_features(X_titanic_df)

# 데이터 분할할
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=0)

C:\Users\selina\AppData\Local\Temp\ipykernel_27596\340051843.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].mean(), inplace=True)
C:\Users\selina\AppData\Local\Temp\ipykernel_27596\340051843.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

In [ ]:
# 모델 생성 & 학습습
myclf = MyDummyClassifier()
myclf.fit(X_train, y_train)

# 모델 예측
my_pred = myclf.predict(X_test)
accuracy_score(y_test, my_pred) #정확도도

0.7877094972067039

In [15]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, my_pred) #row: 실제값, col: 예측측

array([[92, 18],
       [20, 49]])

In [18]:
from sklearn.metrics import precision_score, recall_score

# 정밀도/재현율율
precision_score(y_test, my_pred), recall_score(y_test, my_pred)


(0.7313432835820896, 0.7101449275362319)

# MNIST dataset

In [14]:
from sklearn.datasets import load_digits

digits = load_digits()
digits.data.shape


(1797, 64)

# p.149 MyFakeClassifier 를 이용한 정확도 측정

# 오차행렬

In [20]:
#titanic_df
X_titanic_df.head(), y_titanic_df.head()

(   Pclass  Sex   Age  SibSp  Parch     Fare  Cabin  Embarked
 0       3    1  22.0      1      0   7.2500      7         3
 1       1    0  38.0      1      0  71.2833      2         0
 2       3    0  26.0      0      0   7.9250      7         3
 3       1    0  35.0      1      0  53.1000      2         3
 4       3    1  35.0      0      0   8.0500      7         3,
 0    0
 1    1
 2    1
 3    1
 4    0
 Name: Survived, dtype: int64)

In [28]:
# 전처리리
#X_titanic_df = transform_features(X_titanic_df)
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=11)


In [33]:
# p.156

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred) 
    recall = recall_score(y_test, pred)

    print(confusion)
    print('*'*20)
    print(accuracy, precision, recall)   

In [34]:
# 로지스틱회귀 분류모델 생성
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
lr_clf.predict(X_test)
pred = lr_clf.predict(X_test)

# 정확도, 정밀도, 재현율
get_clf_eval(y_test, pred)

[[104  14]
 [ 13  48]]
********************
0.8491620111731844 0.7741935483870968 0.7868852459016393


c:\Users\selina\.conda\envs\ml_env\lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [37]:
pred_proba = lr_clf.predict_proba(X_test)

In [38]:
pred_proba_result = np.concatenate([pred_proba, pred.reshape(-1,1)],axis=1)
pred_proba_result

array([[0.46235897, 0.53764103, 1.        ],
       [0.87877764, 0.12122236, 0.        ],
       [0.87720996, 0.12279004, 0.        ],
       [0.88243424, 0.11756576, 0.        ],
       [0.85531356, 0.14468644, 0.        ],
       [0.8821124 , 0.1178876 , 0.        ],
       [0.88850554, 0.11149446, 0.        ],
       [0.20887546, 0.79112454, 1.        ],
       [0.7828625 , 0.2171375 , 0.        ],
       [0.36900515, 0.63099485, 1.        ],
       [0.8996726 , 0.1003274 , 0.        ],
       [0.87516493, 0.12483507, 0.        ],
       [0.87720405, 0.12279595, 0.        ],
       [0.88846023, 0.11153977, 0.        ],
       [0.43679356, 0.56320644, 1.        ],
       [0.85911091, 0.14088909, 0.        ],
       [0.90378902, 0.09621098, 0.        ],
       [0.73334014, 0.26665986, 0.        ],
       [0.72465745, 0.27534255, 0.        ],
       [0.17155259, 0.82844741, 1.        ],
       [0.75350245, 0.24649755, 0.        ],
       [0.61874254, 0.38125746, 0.        ],
       [0.

## 이진화 Binarizer

In [ ]:
custom_threshold = 0.4 # 이 기준보다 큰 수치를 생존 아니면 사망
pred_proba_1 = pred_proba[:,1].reshape(-1,1) # 새로운 예측값값 #0.53764103, 0.12122236...

from sklearn.preprocessing import Binarizer
binarizer = Binarizer(threshold = custom_threshold).fit(pred_proba_1) #새로운 예측값으로 이진화한 예측값값
custom_predict = binarizer.transform(pred_proba_1)
get_clf_eval(y_test, custom_predict)

[[98 20]
 [10 51]]
********************
0.8324022346368715 0.7183098591549296 0.8360655737704918


In [ ]:
# get_clf_eval(y_test, pred)
[[104  14]
 [ 13  48]]
********************
0.8491620111731844 0.7741935483870968 0.7868852459016393